In [18]:
from pbpstats.data_loader import StatsNbaEnhancedPbpLoader, StatsNbaPossessionLoader, StatsNbaShotsLoader, StatsNbaGameFinderLoader
from nba_api.stats.endpoints import playercareerstats, playerprofilev2, commonplayerinfo
import os
import psutil
import nba_api
import numpy as np
import json
import os
import time
from glob import glob

In [12]:
season_types = ["Regular Season", "Playoffs"]
seasons = [str(i) + "-" + str(i+1)[-2:] for i in range(2000, 2020)]

all_games = []
for season in seasons:
    for season_type in season_types:
        season_games = StatsNbaGameFinderLoader("nba", season, season_type, "file", "/data")
        for game in season_games.items:
            all_games.append(game.data['game_id'])

In [13]:
print(len(all_games))

25575


In [14]:
all_players = set()
process = psutil.Process(os.getpid())
for i, game_id in enumerate(all_games):
    possesions = StatsNbaPossessionLoader(game_id, "file", "/data")
    for poss in possesions.items:
        for event in poss.data['events']:
            if event.data['event_type'] == 8:
                all_players.add(event.data['player2_id'])
                all_players.add(event.data['player1_id'])
            if event.data['event_type'] == 12:
                for team, players in event.data['period_starters'].items():
                    for player in players:
                        all_players.add(player)

#pbp_loader = StatsNbaEnhancedPbpLoader("0021900001", "file", "/data")
#print(pbp_loader.items[0].data)  # prints dict with the first event of the game

#shot_loader = StatsNbaShotsLoader("0021900001", "file", "/data")
#print(shot_loader.items[0].data) # prints dict with data for one shot from game

In [15]:
print(len(all_players))
outpath = os.path.join(".", "players", "all_players.json")
with open(outpath, "w") as outfile:
    json.dump(list(all_players), outfile)

1969


In [16]:
def myconverter(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime.datetime):
        return obj.__str__()

for player in all_players:
    outpath = os.path.join(".", "players", str(player) + ".json")
    if os.path.exists(outpath):
        continue
    print(player)
    time.sleep(3)
    try:
        info = commonplayerinfo.CommonPlayerInfo(player_id=player, timeout=30)
        data = info.get_data_frames()[0].iloc[0].to_dict()
        with open(outpath, "w") as outfile:
            json.dump(data, outfile, default=myconverter)
    except Exception as e:
        print("Timeout for player", player)

84
85
87
89
93
95
96
98
103
105
109
111
120
121
123
124
128
133
136
137
145
146
147
164
165
166
168
176
177
178
179
180
182
185
190
193
197
198
201
202
208
209
210
213
216
219
221
223
224
228
236
238
239
243
244
248
251
252
255
258
270
271
275
278
280
281
283
292
296
297
299
302
304
317
323
324
330
339
344
345
348
349
353
358
359
361
363
365
369
371
375
376
380
381
383
384
386
389
393
397
399
400
406
417
422
423
426
428
431
435
436
440
441
442
445
446
448
452
458
461
462
467
468
469
672
673
677
679
680
681
682
685
686
688
689
690
692
693
694
695
696
697
699
702
703
708
711
713
714
715
717
718
721
722
724
727
731
735
739
753
754
757
760
761
762
763
764
765
766
779
781
782
788
1628394
891
893
895
896
901
902
101129
904
905
911
913
915
916
919
920
922
923
924
931
932
934
935
937
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
965
966
969
970
976
977
978
979
980
981
983
988
990
994
998
1000
1005
1023
101153
1032
1035
1038
1043
1049
1051
1052
1065
1073
1074
1085
1088
1110
11

In [44]:
player_files = os.path.join(".", "players", "*")
player_files = glob(player_files)
id_slug = {}
for pf in player_files:
    with open(pf, "r") as f:
        player_info = json.load(f)
    if 'PLAYER_SLUG' in player_info:
        slug = player_info['PLAYER_SLUG'].split("-2")[0]
        if slug in id_slug:
            slug = slug + "-2"
            print("DUPLICATE - ", slug)
        player_info['PLAYER_SLUG'] = slug
        with open(pf, "w") as f:
            json.dump(player_info, f)
        id_slug[slug] = player_info['PERSON_ID']
        id_slug[player_info['PERSON_ID']] = slug
    

print(len(all_players))
outpath = os.path.join(".", "players", "all_players.json")
with open(outpath, "w") as outfile:
    json.dump(id_slug, outfile)

DUPLICATE -  dee-brown-2
DUPLICATE -  steven-smith-2
DUPLICATE -  marcus-williams-2
DUPLICATE -  patrick-ewing-2
DUPLICATE -  mike-james-2
DUPLICATE -  chris-johnson-2
DUPLICATE -  chris-wright-2
DUPLICATE -  glen-rice-2
DUPLICATE -  tony-mitchell-2
1969


In [25]:
print(len(player_slugs))

1960


202694
1628369


In [83]:
import yaml

In [118]:
for i, game_id in enumerate(all_games):
    possesions = StatsNbaPossessionLoader(game_id, "file", "/data")
    print(game_id)
    players_on_court = {}
    game_path = os.path.join(".", "possessions", str(game_id))
    os.makedirs(game_path, exist_ok=True)
    for poss_index, poss in enumerate(possesions.items):
        events = []
        poss.data['player_game_fouls'] = poss.data['events'][0].data['player_game_fouls']
        poss.data['fouls_to_give'] = poss.data['events'][0].data['fouls_to_give']
        poss.data['score_start'] = poss.data['events'][0].data['score']
        poss.data['score_end'] = poss.data['events'][-1].data['score']

        for event in poss.data['events']:
            data = event.data
            del data['previous_event']
            del data['next_event']
            del data['fouls_to_give']
            del data['player_game_fouls']
            del data['score']
            #print(data)
            if data['event_type'] == 12:  # Start of game
                players_on_court = {}
                for team, players in data['period_starters'].items():
                    players_on_court[team] = players
            if data['event_type'] == 8:  # Sub in/out
                team_id = data["team_id"]
                sub_out = data["player1_id"]
                sub_in = data["player2_id"]
                players_on_court[team_id].remove(sub_out)
                players_on_court[team_id].append(sub_in)
            events.append(data)
        poss.data['events'] = events
        poss.data['players_on_court'] = players_on_court
        del poss.data['previous_possession']
        del poss.data['next_possession']
        outpath = os.path.join(game_path, str(poss_index).zfill(4) + ".yaml")
        with open(outpath, "w") as f:
            # Ensure json
            d = json.loads(json.dumps(poss.data))
            yaml.dump(d, f)

#pbp_loader = StatsNbaEnhancedPbpLoader("0021900001", "file", "/data")
#print(pbp_loader.items[0].data)  # prints dict with the first event of the game

#shot_loader = StatsNbaShotsLoader("0021900001", "file", "/data")
#print(shot_loader.items[0].data) # prints dict with data for one shot from game

0020000001
0020000002
0020000003
0020000004
0020000005
0020000006
0020000007


KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-114-ae7d833394f8>, line 5)

In [116]:
from nba_api.stats.static import teams
teams = teams.get_teams()

team_id_to_nickname = {}

for t in teams:
    team_id_to_nickname[t['id']] = t['abbreviation']
    team_id_to_nickname[t['abbreviation']] = t['id']

In [123]:
print(team_id_to_nickname)

{1610612737: 'ATL', 'ATL': 1610612737, 1610612738: 'BOS', 'BOS': 1610612738, 1610612739: 'CLE', 'CLE': 1610612739, 1610612740: 'NOP', 'NOP': 1610612740, 1610612741: 'CHI', 'CHI': 1610612741, 1610612742: 'DAL', 'DAL': 1610612742, 1610612743: 'DEN', 'DEN': 1610612743, 1610612744: 'GSW', 'GSW': 1610612744, 1610612745: 'HOU', 'HOU': 1610612745, 1610612746: 'LAC', 'LAC': 1610612746, 1610612747: 'LAL', 'LAL': 1610612747, 1610612748: 'MIA', 'MIA': 1610612748, 1610612749: 'MIL', 'MIL': 1610612749, 1610612750: 'MIN', 'MIN': 1610612750, 1610612751: 'BKN', 'BKN': 1610612751, 1610612752: 'NYK', 'NYK': 1610612752, 1610612753: 'ORL', 'ORL': 1610612753, 1610612754: 'IND', 'IND': 1610612754, 1610612755: 'PHI', 'PHI': 1610612755, 1610612756: 'PHX', 'PHX': 1610612756, 1610612757: 'POR', 'POR': 1610612757, 1610612758: 'SAC', 'SAC': 1610612758, 1610612759: 'SAS', 'SAS': 1610612759, 1610612760: 'OKC', 'OKC': 1610612760, 1610612761: 'TOR', 'TOR': 1610612761, 1610612762: 'UTA', 'UTA': 1610612762, 1610612763:

In [93]:
shot_loader = StatsNbaShotsLoader("0021900001", "file", "/data")
print(shot_loader.items[0].data) # prints dict with data for one shot from game

{'grid_type': 'Shot Chart Detail', 'game_id': '0021900001', 'game_event_id': 10, 'player_id': 1628384, 'player_name': 'OG Anunoby', 'team_id': 1610612761, 'team_name': 'Toronto Raptors', 'period': 1, 'minutes_remaining': 11, 'seconds_remaining': 29, 'event_type': 'Missed Shot', 'action_type': 'Driving Layup Shot', 'shot_type': '2PT Field Goal', 'shot_zone_basic': 'Restricted Area', 'shot_zone_area': 'Center(C)', 'shot_zone_range': 'Less Than 8 ft.', 'shot_distance': 3, 'loc_x': 15, 'loc_y': 28, 'shot_attempted_flag': 1, 'shot_made_flag': 0, 'game_date': '20191022', 'htm': 'TOR', 'vtm': 'NOP'}


In [ ]:
team_id_to_nickname[0] = 'unk'
id_slug[0] = 'unk'
id_slug['0'] = 'unk'

In [124]:
## Game parser
for game in all_games:
    print(game)
    shots = StatsNbaShotsLoader(game, "file", "/data")
    shots = [s.data for s in shots.items]
    event_id_to_shot = {}
    for shot in shots:
        shot['player_id'] = id_slug[shot['player_id']]
        event_id = shot['game_event_id']
        event_id_to_shot[event_id] = shot
    possession_files = glob(os.path.join(".", "possessions", game, "*"))
    for possession_file in possession_files:
        with open(possession_file, "r") as f:
            possession = yaml.load(f)

            new_players_on_court = {}
            for team, players in possession['players_on_court'].items():
                t = team_id_to_nickname[int(team)]
                players = [id_slug[p] for p in players]
                new_players_on_court[t] = players
            possession['players_on_court'] = new_players_on_court

            for event in possession['events']:
                event['team_id'] = team_id_to_nickname[event['team_id']]
                for p_id in ['player1_id', 'player2_id', 'player3_id']
                player1_id: 84
                if event['event_num'] in event_id_to_shot:
                    event['shot_info'] = event_id_to_shot
    print(event)
    break

0020000001
{'clock': '0:00', 'description': '', 'event_action_type': 0, 'event_num': 467, 'event_type': 13, 'game_id': '0020000001', 'non_possession_changing_override': False, 'order': 428, 'period': 4, 'player1_id': 0, 'possession_changing_override': False, 'team_id': 0}


In [126]:
len(id_slug)

3940

In [127]:
len(team_id_to_nickname)

61

In [134]:
team_id_to_nickname
with open("team_ids.yaml", "w") as f:
    yaml.dump(team_id_to_nickname, f)

In [133]:

import json
player_files = os.path.join(".", "players", "*")
player_files = glob(player_files)
all_player_info = {}
for pf in player_files:
    with open(pf, "r") as f:
        player_info = json.load(f)
    if 'PERSON_ID' in player_info:
        all_player_info[player_info['PERSON_ID']] = player_info

with open("player_info.yaml", "w") as f:
    yaml.dump(all_player_info, f)

In [138]:
print(poss.possession_stats)

AttributeError: 'dict' object has no attribute 'event_stats'